In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import tarfile
tar = tarfile.open("/content/drive/MyDrive/Dysarthria/F.tar.bz2", "r:bz2")
tar.extractall()
tar.close()

In [ ]:
tar = tarfile.open("/content/drive/MyDrive/Dysarthria/FC.tar.bz2", "r:bz2")
tar.extractall()
tar.close()

In [ ]:
tar = tarfile.open("/content/drive/MyDrive/Dysarthria/M.tar.bz2", "r:bz2")
tar.extractall()
tar.close()

In [ ]:
tar = tarfile.open("/content/drive/MyDrive/Dysarthria/MC.tar.bz2", "r:bz2")
tar.extractall()
tar.close()

In [ ]:
!pip install pydub

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
F_dy = ["F01/Session1/wav_headMic", "F03/Session1/wav_headMic", "F03/Session2/wav_headMic", "F03/Session3/wav_headMic","F04/Session1/wav_arrayMic", "F04/Session2/wav_headMic"]
F_non = ["FC01/Session1/wav_headMic", "FC02/Session2/wav_arrayMic", "FC02/Session3/wav_headMic", "FC03/Session1/wav_headMic", "FC03/Session2/wav_headMic", "FC03/Session3/wav_headMic"]
M_dy = ["M01/Session1/wav_headMic", "M01/Session2_3/wav_headMic", "M02/Session1/wav_headMic", "M02/Session2/wav_headMic", "M03/Session2/wav_headMic", "M04/Session1/wav_arrayMic", "M04/Session2/wav_headMic", "M05/Session1/wav_headMic", "M05/Session2/wav_headMic"]
M_non = ["MC01/Session1/wav_headMic", "MC01/Session2/wav_headMic", "MC01/Session3/wav_headMic", "MC02/Session1/wav_headMic", "MC02/Session2/wav_headMic", "MC03/Session1/wav_headMic", "MC03/Session2/wav_headMic", "MC04/Session1/wav_headMic", "MC04/Session2/wav_arrayMic"]
dy = F_dy + M_dy
non = F_non + M_non

In [ ]:
import os
dy_files = []
for folder in dy:
    files = os.listdir(folder)
    full_name = []
    for name in files:
      full_name.append(os.path.join(folder, name))
    dy_files += full_name


non_files = []
for folder in non:
    files = os.listdir(folder)
    full_name = []
    for name in files:
      full_name.append(os.path.join(folder, name))
    non_files += full_name

In [ ]:
len(dy_files), len(non_files)

(3323, 5862)

In [ ]:
x = dy_files + non_files
y = 3323*[1] + 5862*[0]

In [ ]:
import numpy as np
x = np.array(x)
y = np.array(y)

def unison_shuffled_copies(a, b):
    assert len(a) == len(b)
    p = np.random.permutation(len(a))
    return a[p], b[p]
x, y = unison_shuffled_copies(x, y)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((7348,), (1837,), (7348,), (1837,))

In [ ]:
import numpy as np
import pandas as pd
import os
import re
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
import librosa
import librosa.display
from sklearn.preprocessing import minmax_scale
import IPython.display as ipd
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve, recall_score
warnings.filterwarnings("ignore")

In [ ]:
from pydub import AudioSegment
from pydub.playback import play
import random

In [ ]:
X_train_final = []
Y_train_final = []
X_test_final = []
Y_test_final = []



In [ ]:
for num, i in tqdm(enumerate(X_train)):
  try:
    a , sr = librosa.load(i)
    mean_mfcc = np.mean(librosa.feature.mfcc(y=a, sr=sr, n_mfcc=128),axis=1)
    X_train_final.append(mean_mfcc)
    Y_train_final.append(y_train[num])
  except EOFError:
    pass

7348it [16:23,  7.47it/s]


In [ ]:
for num, i in tqdm(enumerate(X_test)):
  try:
    a , sr = librosa.load(i)
    mean_mfcc = np.mean(librosa.feature.mfcc(y=a, sr=sr, n_mfcc=128),axis=1)
    X_test_final.append(mean_mfcc)
    Y_test_final.append(y_test[num])
  except EOFError:
    pass

1837it [03:56,  7.77it/s]


In [ ]:
X_train_aug = []
Y_train_aug = []
X_test_aug = []
Y_test_aug = []

In [ ]:
sound = [15, -15]
time = [0.5, 2]
choice = ['s', 't']
for num, i in tqdm(enumerate(X_train)):
  try:
    c = random.choice(choice)
    if c == 's':
      s = random.choice(sound)
      song = AudioSegment.from_wav(i)
      song_new = song + s
      song_new.export("/content/song_new.wav", format='wav')
      a , sr = librosa.load("/content/song_new.wav")
      mean_mfcc = np.mean(librosa.feature.mfcc(y=a, sr=sr, n_mfcc=128),axis=1)
      X_train_aug.append(mean_mfcc)
      Y_train_aug.append(y_train[num])
    else:
      t = random.choice(time)
      a , sr = librosa.load(i)
      song_new = librosa.effects.time_stretch(a, rate=t)
      mean_mfcc = np.mean(librosa.feature.mfcc(y=a, sr=sr, n_mfcc=128),axis=1)
      X_train_aug.append(mean_mfcc)
      Y_train_aug.append(y_train[num])
  except:
    pass

7348it [19:38,  6.23it/s]


In [ ]:
sound = [15, -15]
time = [0.5, 2]
choice = ['s', 't']
X_test_aug = []
for i in tqdm(X_test):
  try:
    c = random.choice(choice)
    if c == 's':
      s = random.choice(sound)
      song = AudioSegment.from_wav(i)
      song_new = song + s
      song_new.export("/content/song_new.wav", format='wav')
      a , sr = librosa.load("/content/song_new.wav")
      mean_mfcc = np.mean(librosa.feature.mfcc(y=a, sr=sr, n_mfcc=128),axis=1)
      X_test_aug.append(mean_mfcc)
      Y_test_aug.append(y_test[num])
    else:
      t = random.choice(time)
      a , sr = librosa.load(i)
      song_new = librosa.effects.time_stretch(a, rate=t)
      mean_mfcc = np.mean(librosa.feature.mfcc(y=a, sr=sr, n_mfcc=128),axis=1)
      X_test_aug.append(mean_mfcc)
      Y_test_aug.append(y_test[num])
  except:
    pass

100%|██████████| 1837/1837 [05:18<00:00,  5.77it/s]


In [ ]:
X_train_final = np.array(X_train_final)
Y_train_final = np.array(Y_train_final)

X_test_final = np.array(X_test_final)
Y_test_final = np.array(Y_test_final)

X_train_aug = np.array(X_train_aug)
Y_train_aug = np.array(Y_train_aug)

X_test_aug = np.array(X_test_aug)
Y_test_aug = np.array(Y_test_aug)

print(X_train_final.shape, Y_train_final.shape, X_test_final.shape, Y_test_final.shape)
print(X_train_aug.shape, Y_train_aug.shape, X_test_aug.shape, Y_test_aug.shape)

(7346, 128) (7346,) (1837, 128) (1837,)
(7346, 128) (7346,) (1837, 128) (0,)


In [ ]:
Y_test_aug = Y_test_final

In [ ]:
print(X_train_final.shape, Y_train_final.shape, X_test_final.shape, Y_test_final.shape)
print(X_train_aug.shape, Y_train_aug.shape, X_test_aug.shape, Y_test_aug.shape)

(7346, 128) (7346,) (1837, 128) (1837,)
(7346, 128) (7346,) (1837, 128) (1837,)


In [ ]:
cd "/content/drive/MyDrive/Dysarthria/augmented_entire"

/content/drive/MyDrive/Dysarthria


In [ ]:
mkdir augmented_entire

In [ ]:
np.save("/content/drive/MyDrive/Dysarthia/X_train_og.npy", X_train_final)
np.save("/content/drive/MyDrive/Dysarthia/y_train_og.npy", Y_train_final)

In [ ]:
np.save("/content/drive/MyDrive/Dysarthria/augmented_entire/X_train_final.npy", X_train_final)
np.save("/content/drive/MyDrive/Dysarthria/augmented_entire/Y_train_final.npy", Y_train_final)
np.save("/content/drive/MyDrive/Dysarthria/augmented_entire/X_test_final.npy", X_test_final)
np.save("/content/drive/MyDrive/Dysarthria/augmented_entire/Y_test_final.npy", Y_test_final)
np.save("/content/drive/MyDrive/Dysarthria/augmented_entire/X_train_aug.npy", X_train_aug)
np.save("/content/drive/MyDrive/Dysarthria/augmented_entire/Y_train_aug.npy", Y_train_aug)
np.save("/content/drive/MyDrive/Dysarthria/augmented_entire/X_test_aug.npy", X_test_aug)
np.save("/content/drive/MyDrive/Dysarthria/augmented_entire/Y_test_aug.npy", Y_test_aug)


In [ ]:
ls "/content/drive/MyDrive/Dysarthria/augmented_entire/"

X_test_aug.npy    X_train_aug.npy    Y_test_aug.npy    Y_train_aug.npy
X_test_final.npy  X_train_final.npy  Y_test_final.npy  Y_train_final.npy


In [ ]:
for num, i in tqdm(enumerate(X_test)):
  try:
    a , sr = librosa.load(i)
    mean_mfcc = np.mean(librosa.feature.mfcc(y=a, sr=sr, n_mfcc=128),axis=1)
    X_test_final.append(mean_mfcc)
    Y_test_final.append(y_test[num])
  except EOFError:
    pass

100%|██████████| 1837/1837 [03:51<00:00,  7.92it/s]


((7346, 128), (7346,))

In [ ]:
np.save("/content/drive/MyDrive/Dysarthia/X_test_og.npy", X_test_final)
np.save("/content/drive/MyDrive/Dysarthia/y_test_og.npy", Y_test_final)

In [ ]:
sound = [15, -15]
time = [0.5, 2]
choice = ['s', 't']
X_test_aug = []
for i in tqdm(X_test):
  try:
    c = random.choice(choice)
    if c == 's':
      s = random.choice(sound)
      song = AudioSegment.from_wav(i)
      song_new = song + s
      song_new.export("/content/song_new.wav", format='wav')
      a , sr = librosa.load("/content/song_new.wav")
      mean_mfcc = np.mean(librosa.feature.mfcc(y=a, sr=sr, n_mfcc=128),axis=1)
      X_test_aug.append(mean_mfcc)
    else:
      t = random.choice(time)
      a , sr = librosa.load(i)
      song_new = librosa.effects.time_stretch(a, rate=t)
      mean_mfcc = np.mean(librosa.feature.mfcc(y=a, sr=sr, n_mfcc=128),axis=1)
      X_test_aug.append(mean_mfcc)
  except EOFError:
    pass

100%|██████████| 1837/1837 [04:38<00:00,  6.60it/s]


In [ ]:
len(X_test_aug)

1837

In [ ]:
np.save("/content/drive/MyDrive/Dysarthia/X_test_aug.npy", X_test_aug)
np.save("/content/drive/MyDrive/Dysarthia/y_test_aug.npy", y_test)

In [ ]:
ls "/content/drive/MyDrive/Dysarthia/"

 count.ipynb                                                   M.tar.bz2
 data_enitre.csv                                               preprocess.ipynb
 dysarthia-classification-using-cnns-on-audios.ipynb           X.npy
 Dysarthia.gdoc                                                X_test_aug.npy
 Dysarthria.gsheet                                             X_test_og.npy
 FC.tar.bz2                                                    X_train_og.npy
 features.csv                                                  y.npy
 F.tar.bz2                                                     y_test_aug.npy
 MC.tar.bz2                                                    y_test_og.npy
'MFCC Feature Extraction and Exploratory Data Analysis.gdoc'   y_train_og.npy
